In [1]:
# OSFM examples
# With DOI
# Without DOI
# OpenAlex examples
# With DOI : https://api.openalex.org/works/https://doi.org/10.35562/arabesques.3084
# Without DOI : https://api.openalex.org/works/W3204678669

In [2]:
from dotenv import load_dotenv
import os
import pymongo
import re
import requests
from retry import retry

In [3]:
load_dotenv()

FOSM_BASE_URL = "https://cluster.elasticsearch.dataesr.ovh"
FOSM_INDEX = "bso-publications"
FOSM_LIMIT = 0  # Set to 0 for no limit
FOSM_PAGE_SIZE = 10000  # Maximum is 10000
FOSM_PIT_KEEP_ALIVE = "5m"
MONGO_DB = "bsocoverage"
MONGO_COLLECTION = "publications"
OA_LIMIT = 0  # Set to 0 for no limit
OA_PAGE_SIZE = 200  # Maximum is 200

# Access the environment variables from the .env file
FOSM_AUTHORIZATION=os.getenv("FOSM_AUTHORIZATION")
MONGO_URI=os.getenv("MONGO_URI", "mongodb://localhost:27017/")
OA_API_KEY=os.getenv("OA_API_KEY")

In [4]:
mongo_database = pymongo.MongoClient(MONGO_URI)[MONGO_DB]
mongo_collection = mongo_database[MONGO_COLLECTION]
# mongo_collection.drop()
mongo_collection.create_index([("id", pymongo.ASCENDING)], unique=True)

'id_1'

In [5]:
# Number of publications in French OSM
json = {
  "query": { "bool": { "must": [
    { "range": { "year": { "gte": 2013, "lte": 2021 } } },
    { "term": { "bso_country_corrected": "fr" } },
    { "terms": { "genre.keyword": [ "journal-article", "proceedings", "book-chapter", "book", "preprint" ] } },
  ] } },
}
r = requests.get("/".join([FOSM_BASE_URL, FOSM_INDEX, "_count"]), headers={"Authorization": f"Basic {FOSM_AUTHORIZATION}"}, json=json)
fosm_total_count = r.json().get("count")
fosm_total_count

1826102

In [6]:
@retry(Exception, tries=5, delay=10)
def get_fosm_publications(pit, total_results_count=0, search_after=None):
    try:
        print(f"\"{pit}\", {total_results_count}, {search_after}")
        json = {
            "pit": {"id":  pit, "keep_alive": FOSM_PIT_KEEP_ALIVE},
            "query": { "bool": { "must": [
                { "range": { "year": { "gte": 2013, "lte": 2021 } } },
                { "term": { "bso_country_corrected": "fr" } },
                { "terms": { "genre.keyword": [ "journal-article", "proceedings", "book-chapter", "book", "preprint" ] } },
            ] } },
            "size": FOSM_PAGE_SIZE,
            "sort": ["_shard_doc"],
        }
        if search_after:
            json["search_after"] = search_after
            json["track_total_hits"] = False
        r = requests.get("/".join([FOSM_BASE_URL, "_search"]),
                        headers={"Authorization": f"Basic {FOSM_AUTHORIZATION}"}, json=json)
        response = r.json()
        results = response.get("hits").get("hits")
        actions = []
        for publication in results:
            doi = publication.get("_source", {}).get("doi")
            hal_id = publication.get("_source", {}).get("hal_id")
            id = doi if doi else hal_id
            id = id.lower()
            publication = {
                "all_ids": publication.get("_source").get("external_ids"),
                "id": id,
                "is_in_fosm": True,
                "year_fosm": publication.get("_source").get("year"),
                "fosm": publication.get("_source"),
            }
            if doi:
                publication["doi"] = doi
            if hal_id:
                publication["hal_id"] = hal_id
            actions.append(pymongo.UpdateOne(
                {"id": id}, {"$set": publication}, upsert=True))
        if len(actions) > 0:
            mongo_collection.bulk_write(actions, ordered=False)
        results_count = len(results)
        total_results_count += results_count
        last_result = results[results_count - 1]
        next_pit = response.get("pit_id")
        del json
        del r
        del response
        del results
        del actions
        print('{:.0f} %'.format((total_results_count / fosm_total_count) * 100))
        if results_count > 0 and (FOSM_LIMIT == 0 or total_results_count < FOSM_LIMIT):
            search_after = last_result.get("sort")
            return get_fosm_publications(next_pit, total_results_count, search_after)
        else:
            return total_results_count
    except Exception as error:
        print(error)
        raise

In [7]:
# Delete all documents from FOSM only, in Mongo
# delete = mongo_collection.delete_many({ "is_in_fosm": "true", "is_in_openalex": { "$exists": False } })
# print(delete.deleted_count, " documents deleted")
# Delete "is_in_fosm" field for all documents in Mongo
# updated = mongo_collection.update_many({}, { "$unset": { "is_in_fosm": 1 } } , True)
# print(updated.modified_count, " documents modified")
# Get Point In Time
r = requests.post("/".join([FOSM_BASE_URL, FOSM_INDEX, f"_pit?keep_alive={FOSM_PIT_KEEP_ALIVE}"]), headers={"Authorization": f"Basic {FOSM_AUTHORIZATION}"})
pit = r.json().get("id")
# Collect all publications with DOI in French OSM
fosm_publications = get_fosm_publications(pit)
print(fosm_publications)
# Delete Point In Time
r = requests.delete("/".join([FOSM_BASE_URL, "_pit"]), headers={"Authorization": f"Basic {FOSM_AUTHORIZATION}"}, json={"id": pit})

"48myAwEZYnNvLXB1YmxpY2F0aW9ucy0yMDIzMTAwNhZnYkRyMENLWFRsdUtOOUJfazY0ZV9RABZDWTQ0M0VMQ1NRV08wR1FFWmxEdWhnAAAAAAAHY12uFkxrMTJQQkNHVGRPYUFRQ3gxUVllbmcAARZnYkRyMENLWFRsdUtOOUJfazY0ZV9RAAA=", 0, None
1 %
"48myAwEZYnNvLXB1YmxpY2F0aW9ucy0yMDIzMTAwNhZnYkRyMENLWFRsdUtOOUJfazY0ZV9RABZDWTQ0M0VMQ1NRV08wR1FFWmxEdWhnAAAAAAAHY12uFkxrMTJQQkNHVGRPYUFRQ3gxUVllbmcAARZnYkRyMENLWFRsdUtOOUJfazY0ZV9RAAA=", 10000, [43628]
1 %
"48myAwEZYnNvLXB1YmxpY2F0aW9ucy0yMDIzMTAwNhZnYkRyMENLWFRsdUtOOUJfazY0ZV9RABZDWTQ0M0VMQ1NRV08wR1FFWmxEdWhnAAAAAAAHY12uFkxrMTJQQkNHVGRPYUFRQ3gxUVllbmcAARZnYkRyMENLWFRsdUtOOUJfazY0ZV9RAAA=", 20000, [80884]
2 %
"48myAwEZYnNvLXB1YmxpY2F0aW9ucy0yMDIzMTAwNhZnYkRyMENLWFRsdUtOOUJfazY0ZV9RABZDWTQ0M0VMQ1NRV08wR1FFWmxEdWhnAAAAAAAHY12uFkxrMTJQQkNHVGRPYUFRQ3gxUVllbmcAARZnYkRyMENLWFRsdUtOOUJfazY0ZV9RAAA=", 30000, [107339]
2 %
"48myAwEZYnNvLXB1YmxpY2F0aW9ucy0yMDIzMTAwNhZnYkRyMENLWFRsdUtOOUJfazY0ZV9RABZDWTQ0M0VMQ1NRV08wR1FFWmxEdWhnAAAAAAAHY12uFkxrMTJQQkNHVGRPYUFRQ3gxUVllbmcAARZnYkRyMENLWFRsdUtOOUJfazY0Z

KeyboardInterrupt: 

In [8]:
# Number of French publications in OpenAlex
r = requests.get(
    f"https://api.openalex.org/works?filter=institutions.country_code:FR,is_paratext:false,publication_year:2013-2021&api_key={OA_API_KEY}")
openalex_total_count = r.json().get("meta").get("count")
print(openalex_total_count)

1902322


In [9]:
@retry(Exception, tries=5, delay=30)
def get_openalex_publications(cursor, total_results_count):
    print(f"\"{cursor}\", {total_results_count}")
    r = requests.get(
        f"https://api.openalex.org/works?filter=institutions.country_code:FR,is_paratext:false,publication_year:2013-2021&per-page={OA_PAGE_SIZE}&api_key={OA_API_KEY}&cursor={cursor}")
    response = r.json()
    results = response.get("results")
    actions = []
    for publication in results:
        open_alex_id = publication.get("id")
        doi = False
        if publication.get("doi"):
            doi = publication.get("doi", "").replace("https://doi.org/", "")
        hal_landing_page_urls = [location.get("landing_page_url") for location in response.get("locations", []) if re.match("^https:\/\/hal\.(science|archives-ouvertes\.fr|inria\.fr)\/(hal-\d*)", location.get("landing_page_url", ""))]
        hal_ids_uniq = list(set([hal_landing_page_url.split('/')[3] for hal_landing_page_url in hal_landing_page_urls]))
        if len(hal_ids_uniq) > 1:
            print(f"More than one hal_id in OpenAlex work : {open_alex_id}")
            hal_id = False
        else:
            hal_id = hal_ids_uniq[0] if len(hal_ids_uniq) == 1 else False
        id = doi if doi else hal_id if hal_id else open_alex_id
        id = id.lower()
        if id:
            all_ids = [{"id_type": k, "id_value": v} for k, v in publication.get("ids").items()]
            if open_alex_id and len([id for id in all_ids if id.get("id_type") == "openalex"]) == 0:
                all_ids.append({"id_type": "openalex", "id_value": open_alex_id})
            if doi and len([id for id in all_ids if id.get("id_type") == "doi"]) == 0:
                all_ids.append({"id_type": "doi", "id_value": doi})
            if hal_id and len([id for id in all_ids if id.get("id_type") == "hal_id"]) == 0:
                all_ids.append({"id_type": "hal_id", "id_value": hal_id})
            publication = {
                "all_ids": all_ids,
                "id": id,
                "is_in_openalex": True,
                "year_openalex": publication.get("publication_year"),
                "openalex": publication,
            }
            actions.append(pymongo.UpdateOne(
                {"id": id}, {"$set": publication}, upsert=True))
    if len(actions) > 0:
        mongo_collection.bulk_write(actions, ordered=False)
    results_count = len(results)
    total_results_count += results_count
    next_cursor = response.get("meta").get("next_cursor")
    del actions
    del r
    del response
    del results
    print('{:.0f} %'.format((total_results_count / openalex_total_count) * 100))
    if next_cursor is not None and results_count > 0 and (OA_LIMIT == 0 or len(total_results_count) < OA_LIMIT):
        return get_openalex_publications(next_cursor, total_results_count)
    else:
        return total_results_count

In [16]:
# Delete all documents from OpenAlex only, in Mongo
# delete = mongo_collection.delete_many({ "is_in_openalex": "true", "is_in_fosm": { "$exists": False } })
# print(delete.deleted_count, " documents deleted")
# Delete "is_in_openalex" field for all documents in Mongo
# updated = mongo_collection.update_many({}, { "$unset": { "is_in_openalex": 1 } } , { "multi": True })
# print(updated.updated_count, " documents updated")
# Collect all French publications in OpenAlex
# openalex_publications = get_openalex_publications("*", 0)
openalex_publications = get_openalex_publications("IlsyLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzI5MTM2MTAzNzUnXSI=", 736600)
print(openalex_publications)

"IlsyLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzI5MTM2MTAzNzUnXSI=", 736600
39 %
"IlsyLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzI5MTUwMTgxMDcnXSI=", 736800
39 %
"IlsyLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzI5MTc5MzczNzQnXSI=", 737000
39 %
"IlsyLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzI5MjEyMTcxNTYnXSI=", 737200
39 %
"IlsyLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzI5MjI3MjI0NjUnXSI=", 737400
39 %
"IlsyLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzI5Mjc0OTY1NDcnXSI=", 737600
39 %
"IlsyLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzI5MzYyMzE0MzInXSI=", 737800
39 %
"IlsyLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzI5Mzk3NzQ4MTknXSI=", 738000
39 %
"IlsyLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzI5NDI4MTEwMzMnXSI=", 738200
39 %
"IlsyLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzI5NDQ2NTk0NjYnXSI=", 738400
39 %
"IlsyLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzI5NDU3NzE0NzgnXSI=", 738600
39 %
"IlsyLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzI5NDcwNTIzMDknXSI=", 738800
39 %
"IlsyLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzI5NDg1NDI0OTMnXSI=", 739000
39 %
"IlsyLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzI5NDk4MTQ2ODInXSI=", 7392

In [ ]:
# Dump the whole database
# mongodump --uri="mongodb://localhost:27017" --archive=bsocoverage.20230917.gz --gzip --db=bsocoverage
# Restore a gzipped database
# mongorestore --uri="mongodb://localhost:27017" --archive=bsocoverage.20230917.gz --gzip